In [4]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import csv
import os

AttributeError: module 'torch' has no attribute 'version'

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

In [ ]:
class FastDataset(Dataset):
    def __init__(self, data, labels, num_classes):
        self.dataset = data
        self.labels = labels
        self.number_classes = num_classes
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        return self.dataset[index], self.labels[index]

In [ ]:
# wersja bez augmentacji
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# wersja z augmentacją
transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(0.2),
    transforms.RandomVerticalFlip(0.2),
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# wersja z augmentacją
transform3 = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [5]:
transform = transform1

dataset = datasets.ImageFolder("train/", transform=transform)
classes = dataset.classes
train_loader = DataLoader(dataset, batch_size=len(dataset))
data_train = next(iter(train_loader))
fast_dataset_train = FastDataset(data_train[0], data_train[1], num_classes=50)
fast_train_loader = DataLoader(fast_dataset_train, batch_size = 128)

In [5]:
transform = transform1

dataset = datasets.ImageFolder("train/", transform=transform)
train_size = int(0.8 * len(dataset)) 
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
classes = dataset.classes

train_loader = DataLoader(train_set, batch_size=len(train_set))
data_train = next(iter(train_loader))
fast_dataset_train = FastDataset(data_train[0], data_train[1], num_classes=50)
fast_train_loader = DataLoader(fast_dataset_train, batch_size = 128)

test_loader = DataLoader(test_set, batch_size=128, shuffle=True, num_workers=12)
data_test = next(iter(test_loader))
fast_dataset_test = FastDataset(data_test[0], data_test[1], num_classes=50)
fast_test_loader = DataLoader(fast_dataset_test, batch_size = 128)

In [6]:
# accuracy 50%
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(192)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.pool3 = nn.AvgPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 8 * 8, 2048)
        self.dropout1 = nn.Dropout(0.4)  
        self.fc2 = nn.Linear(2048, 2048)
        self.dropout2 = nn.Dropout(0.4)  
        self.fc3 = nn.Linear(2048, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn5(self.conv5(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x))))))))))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after the first fully connected layer
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after the second fully connected layer
        x = self.fc3(x)
        return x


net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_run

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.001)

In [8]:
def test_accuracy(net, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for (images, labels) in test_loader:
            images = images.to(device)
            outputs = net(images).cpu()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total * 100

In [ ]:
# trening z monitorowaniem accuracy
test_accuracy_history = []
train_loss_history = []

for epoch in range(25):
    running_loss = 0.0
    for (inputs, labels) in fast_train_loader:
        net.train()
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # net.eval()
    # test_acc = test_accuracy(net, fast_test_loader)
    # test_accuracy_history.append(test_acc)
    net.train()
    train_loss = running_loss / len(fast_train_loader)
    train_loss_history.append(train_loss)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.7f}, Test Accuracy: {1:.3f}%")
print('Finished Training')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.plot(test_accuracy_history)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Accuracy over Epochs')

ax2.plot(train_loss_history)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Train Loss')
ax2.set_title('Train Loss over Epochs')

plt.tight_layout()
plt.show()

In [ ]:
print('Accuracy of the network on the test images: %.3f %%' % (test_accuracy(net, fast_test_loader)))

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in test_loader:
        images, labels = data    
        images = images.to(device)
        outputs = net(images).cpu()   
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,accuracy))

# Trenowanie modelu finalnego od nowa na wszystkim


In [11]:
dataset = datasets.ImageFolder("train/", transform=transform)
train_size = int(len(dataset)-1) 
test_size = 1
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
classes = dataset.classes

train_loader = DataLoader(train_set, batch_size=len(train_set))
data_train = next(iter(train_loader))
fast_dataset_train = FastDataset(data_train[0], data_train[1], num_classes=50)
fast_train_loader = DataLoader(fast_dataset_train, batch_size = 128)

In [12]:
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
    running_loss = 0.0
    for (inputs, labels) in fast_train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(fast_dataset_train):.7f}")
print('Finished Training')

Epoch 1, Loss: 0.0262797
Epoch 2, Loss: 0.0225023
Epoch 3, Loss: 0.0204499
Epoch 4, Loss: 0.0189021
Epoch 5, Loss: 0.0175066
Epoch 6, Loss: 0.0161496
Epoch 7, Loss: 0.0149356
Epoch 8, Loss: 0.0137359
Epoch 9, Loss: 0.0126461
Epoch 10, Loss: 0.0115783
Finished Training


In [13]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_all_data = datasets.ImageFolder("test_all/", transform=test_transform)
test_all_loader = torch.utils.data.DataLoader(test_all_data, batch_size=128, shuffle=False, num_workers=12)

predictions = []
with torch.no_grad():
    for images, _ in test_all_loader:
        images = images.to(device)
        outputs = net(images).cpu()
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.tolist())

file_names = [os.path.basename(file_path) for file_path, _ in test_all_data.imgs]

with open("grądziel_misztal.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for file_name, prediction in zip(file_names, predictions):
        writer.writerow([file_name, prediction])